In [30]:
import pandas as pd
import os
import numpy as np
import random
import matplotlib.pyplot as plt

In [45]:
data_wall = pd.read_csv('/Users/qwert/Documents/TASKAR/Jupyter_Notebooks/df_wall_final.csv',sep=';')
data_floor = pd.read_csv('/Users/qwert/Documents/TASKAR/Jupyter_Notebooks/df_floor_final.csv',sep=';')


In [46]:
data_wall.head(2)

,x,y,z
0,0.999975,0.999949,0.973167
1,0.999992,0.999984,0.970089


In [47]:
data_floor.head(2)

,x,y,z
0,0.999997,0.999971,0.969438
1,0.999998,0.999971,0.969314


In [11]:
maze_width = 10
maze_height = 10


In [14]:
maze = np.ones((maze_height, maze_width), dtype=int)


In [15]:
points = [(1, 2), (3, 4), (5, 6)]  # Replace with your actual points

for x, y in points:
    maze[y, x] = 0


In [18]:
def recursive_backtracking(maze, x, y):
    # Define movement directions (up, down, left, right)
    directions = [(0, -2), (0, 2), (-2, 0), (2, 0)]

    # Shuffle the directions randomly
    random.shuffle(directions)

    for dx, dy in directions:
        new_x, new_y = x + dx, y + dy

        if 0 <= new_x < maze_width and 0 <= new_y < maze_height and maze[new_y, new_x] == 1:
            # Remove the wall between the current cell and the new cell
            maze[new_y, new_x] = 0
            maze[y + dy // 2, x + dx // 2] = 0
            recursive_backtracking(maze, new_x, new_y)

# Start the maze generation process from a random cell
start_x, start_y = random.randrange(1, maze_width, 2), random.randrange(1, maze_height, 2)
recursive_backtracking(maze, start_x, start_y)


In [19]:
for row in maze:
    print(" ".join(["#" if cell == 1 else " " for cell in row]))


# # # # # # # # # #
#       #          
#   #   #   # # #  
#   #           #  
#   #   # # # # #  
#       #       #  
# # # # #   #   #  
#           #   #  
#   # # # # # # #  
#                  


In [48]:
data_floor['x'].min(), data_floor['x'].max()

(0.9999171099876456, 1.000001355007604)

In [49]:
data_floor['y'].min(), data_floor['y'].max()

(0.9997980151110956, 0.9999982420971666)

In [50]:
data_floor['x'][2] - data_floor['x'][1]

-5.206504736787743e-06

In [65]:
import numpy as np
import plotly.graph_objects as go

# Define grid boundaries and resolution
x_min, x_max = 0.9999190418890664, 1.0
y_min, y_max = 0.999839295640645, 1.0
resolution = data_floor['x'][1] - data_floor['x'][2]

# Calculate the number of grid points
num_x_points = int(np.ceil((x_max - x_min) / resolution))
num_y_points = int(np.ceil((y_max - y_min) / resolution))

# Generate the grid
grid_x = []
grid_y = []
for i in range(num_x_points):
    for j in range(num_y_points):
        x_value = x_min + i * resolution
        y_value = y_min + j * resolution
        grid_x.append(x_value)
        grid_y.append(y_value)

# Create a scatter plot for the grid
grid_trace = go.Scatter(
    x=grid_x,
    y=grid_y,
    mode='markers',
    marker=dict(size=2, color='lightgray')
)

# Create layout
layout = go.Layout(
    title='Grid of Points',
    xaxis=dict(title='X'),
    yaxis=dict(title='Y', side='left', showgrid=True),  # Left y-axis
    yaxis2=dict(title='Y2', side='right', overlaying='y', showgrid=True),  # Right y-axis
)

# Create a figure
fig = go.Figure(data=[grid_trace], layout=layout)

# Show the plot
fig.show()


In [63]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd

# Define grid boundaries and resolution
x_min, x_max = data_floor['x'].min(), data_floor['x'].max()
y_min, y_max = data_floor['y'].min(), data_floor['y'].max()
resolution = data_floor['x'][1] - data_floor['x'][2]

# Calculate the number of grid points
num_x_points = int(np.ceil((x_max - x_min) / resolution))
num_y_points = int(np.ceil((y_max - y_min) / resolution))

# Generate the grid
grid_x = []
grid_y = []
for i in range(num_x_points):
    for j in range(num_y_points):
        x_value = x_min + i * resolution
        y_value = y_min + j * resolution
        grid_x.append(x_value)
        grid_y.append(y_value)

# Create a DataFrame for grid points
grid_df = pd.DataFrame({'X': grid_x, 'Y': grid_y})

# Define some sample 'x' and 'y' points
sample_df = data_floor[['x', 'y']]  # Example 'x' and 'y' points

# Create a DataFrame for obstacle positions
obstacle_df = data_wall[['x', 'y']]

# Create a scatter plot for the grid
grid_trace = go.Scatter(
    x=grid_df['X'],
    y=grid_df['Y'],
    mode='markers',
    marker=dict(size=2, color='lightgray'),
    name='Grid Points',
    yaxis='y1'  # Associate with the first y-axis
)

# Create a scatter plot for the sample 'x' and 'y' points
sample_trace = go.Scatter(
    x=sample_df['x'],
    y=sample_df['y'],
    mode='markers',
    marker=dict(size=10, color='blue', symbol='star'),
    name='Floor Points',
    yaxis='y1'  # Associate with the first y-axis
)

# Create a scatter plot for obstacle cells
obstacle_trace = go.Scatter(
    x=obstacle_df['x'],
    y=obstacle_df['y'],
    mode='markers',
    marker=dict(size=10, color='red', symbol='square'),
    name='Wall',
    yaxis='y1'  # Associate with the first y-axis
)

# Create layout
layout = go.Layout(
    title='Grid of Points with Floor Points and Obstacles',
    xaxis=dict(title='X'),
    yaxis=dict(title='Y', side='left', showgrid=True),
    showlegend=True
)

# Create a figure and add traces
fig = go.Figure(data=[grid_trace, sample_trace, obstacle_trace], layout=layout)

# Show the plot
fig.show()


In [87]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import networkx as nx

# Define grid boundaries and resolution
x_min, x_max = data_floor['x'].min(), data_floor['x'].max()
y_min, y_max = data_floor['y'].min(), data_floor['y'].max()
resolution = data_floor['x'][1] - data_floor['x'][2]

# Calculate the number of grid points
num_x_points = int(np.ceil((x_max - x_min) / resolution))
num_y_points = int(np.ceil((y_max - y_min) / resolution))

# Generate the grid
grid_x = []
grid_y = []
for i in range(num_x_points):
    for j in range(num_y_points):
        x_value = x_min + i * resolution
        y_value = y_min + j * resolution
        grid_x.append(x_value)
        grid_y.append(y_value)

# Create a DataFrame for grid points
grid_df = pd.DataFrame({'X': grid_x, 'Y': grid_y})

# Define some sample 'x' and 'y' points
sample_df = data_floor[['x', 'y']]  # Example 'x' and 'y' points

# Create a DataFrame for obstacle positions
obstacle_df = data_wall[['x', 'y']]

# Create a scatter plot for the grid
grid_trace = go.Scatter(
    x=grid_df['X'],
    y=grid_df['Y'],
    mode='markers',
    marker=dict(size=2, color='lightgray'),
    name='Grid Points',
    yaxis='y1'  # Associate with the first y-axis
)

# Create a scatter plot for the sample 'x' and 'y' points
sample_trace = go.Scatter(
    x=sample_df['x'],
    y=sample_df['y'],
    mode='markers',
    marker=dict(size=10, color='green', symbol='star'),
    name='Floor Points',
    yaxis='y1'  # Associate with the first y-axis
)

# Create a scatter plot for obstacle cells
obstacle_trace = go.Scatter(
    x=obstacle_df['x'],
    y=obstacle_df['y'],
    mode='markers',
    marker=dict(size=10, color='red', symbol='square'),
    name='Wall',
    yaxis='y1'  # Associate with the first y-axis
)

# Create layout
layout = go.Layout(
    title='Grid of Points with Floor Points and Obstacles',
    xaxis=dict(title='X'),
    yaxis=dict(title='Y', side='left', showgrid=True),
    showlegend=True
)

# Create a figure and add traces
fig = go.Figure(data=[grid_trace, sample_trace, obstacle_trace], layout=layout)

# Define the start and end points
start_point = (1, 0.9998)
end_point = (0.999927, 0.999844)

# Add start and end points to the figure
fig.add_trace(go.Scatter(x=[start_point[0]], y=[start_point[1]], mode='markers', marker=dict(size=10, color='green'), name='Start'))
fig.add_trace(go.Scatter(x=[end_point[0]], y=[end_point[1]], mode='markers', marker=dict(size=10, color='purple'), name='End'))

# Now, implement the A* algorithm to find the path between start and end points
# Update the 'path' variable with your calculated path

# Assuming 'path' is a list of (x, y) coordinates representing the path
path = [(1, 0.9998), (1.001, 0.99985), (0.999927, 0.999844)]  # Replace with your actual path

# Create a trace for the path
path_trace = go.Scatter(
    x=[point[0] for point in path],
    y=[point[1] for point in path],
    mode='lines',
    line=dict(color='cyan', width=2),
    name='Path'
)

# Add the path trace to the figure
fig.add_trace(path_trace)

# Show the plot
fig.show()


In [85]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import networkx as nx

# Define grid boundaries and resolution
x_min, x_max = data_floor['x'].min(), data_floor['x'].max()
y_min, y_max = data_floor['y'].min(), data_floor['y'].max()
resolution = data_floor['x'][1] - data_floor['x'][2]

# Calculate the number of grid points
num_x_points = int(np.ceil((x_max - x_min) / resolution))
num_y_points = int(np.ceil((y_max - y_min) / resolution))

# Generate the grid
grid_x = []
grid_y = []
for i in range(num_x_points):
    for j in range(num_y_points):
        x_value = x_min + i * resolution
        y_value = y_min + j * resolution
        grid_x.append(x_value)
        grid_y.append(y_value)

# Create a DataFrame for grid points
grid_df = pd.DataFrame({'X': grid_x, 'Y': grid_y})

# Define some sample 'x' and 'y' points
sample_df = data_floor[['x', 'y']]  # Example 'x' and 'y' points

# Create a DataFrame for obstacle positions
obstacle_df = data_wall[['x', 'y']]

# Create a scatter plot for the grid
grid_trace = go.Scatter(
    x=grid_df['X'],
    y=grid_df['Y'],
    mode='markers',
    marker=dict(size=2, color='lightgray'),
    name='Grid Points',
    yaxis='y1'  # Associate with the first y-axis
)

# Create a scatter plot for the sample 'x' and 'y' points
sample_trace = go.Scatter(
    x=sample_df['x'],
    y=sample_df['y'],
    mode='markers',
    marker=dict(size=10, color='green', symbol='star'),
    name='Floor Points',
    yaxis='y1'  # Associate with the first y-axis
)

# Create a scatter plot for obstacle cells
obstacle_trace = go.Scatter(
    x=obstacle_df['x'],
    y=obstacle_df['y'],
    mode='markers',
    marker=dict(size=5, color='red', symbol='square'),
    name='Wall',
    yaxis='y1'  # Associate with the first y-axis
)

# Create layout
layout = go.Layout(
    title='Grid of Points with Floor Points and Obstacles',
    xaxis=dict(title='X'),
    yaxis=dict(title='Y', side='left', showgrid=True),
    showlegend=True
)

# Create a figure and add traces
fig = go.Figure(data=[grid_trace, sample_trace, obstacle_trace], layout=layout)

# Define the start and end points (find nearest nodes)
start_point = (1, 0.9998)
end_point = (0.999927, 0.999844)

def find_nearest_node(graph, point):
    nearest_node = None
    min_distance = float('inf')
    for node in graph.nodes:
        distance = np.linalg.norm(np.array(node) - np.array(point))
        if distance < min_distance:
            nearest_node = node
            min_distance = distance
    return nearest_node

# Find nearest nodes
start_node = find_nearest_node(G, start_point)
end_node = find_nearest_node(G, end_point)

# Add start and end points to the figure
fig.add_trace(go.Scatter(x=[start_node[0]], y=[start_node[1]], mode='markers', marker=dict(size=10, color='green'), name='Start'))
fig.add_trace(go.Scatter(x=[end_node[0]], y=[end_node[1]], mode='markers', marker=dict(size=10, color='purple'), name='End'))

# Use NetworkX's A* algorithm to find the path
path = nx.astar_path(G, start_node, end_node)

# Create a trace for the path
path_trace = go.Scatter(
    x=[point[0] for point in path],
    y=[point[1] for point in path],
    mode='lines',
    line=dict(color='cyan', width=2),
    name='Path'
)

# Add the path trace to the figure
fig.add_trace(path_trace)

# Show the plot
fig.show()


In [84]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import networkx as nx

# Define grid boundaries and resolution
x_min, x_max = data_floor['x'].min(), data_floor['x'].max()
y_min, y_max = data_floor['y'].min(), data_floor['y'].max()
resolution = data_floor['x'][1] - data_floor['x'][2]

# Calculate the number of grid points
num_x_points = int(np.ceil((x_max - x_min) / resolution))
num_y_points = int(np.ceil((y_max - y_min) / resolution))

# Generate the grid
grid_x = []
grid_y = []
for i in range(num_x_points):
    for j in range(num_y_points):
        x_value = x_min + i * resolution
        y_value = y_min + j * resolution
        grid_x.append(x_value)
        grid_y.append(y_value)

# Create a DataFrame for grid points
grid_df = pd.DataFrame({'X': grid_x, 'Y': grid_y})

# Define some sample 'x' and 'y' points
sample_df = data_floor[['x', 'y']]  # Example 'x' and 'y' points

# Create a DataFrame for obstacle positions
obstacle_df = data_wall[['x', 'y']]

# Create a scatter plot for the grid
grid_trace = go.Scatter(
    x=grid_df['X'],
    y=grid_df['Y'],
    mode='markers',
    marker=dict(size=2, color='lightgray'),
    name='Grid Points',
    yaxis='y1'  # Associate with the first y-axis
)

# Create a scatter plot for the sample 'x' and 'y' points
sample_trace = go.Scatter(
    x=sample_df['x'],
    y=sample_df['y'],
    mode='markers',
    marker=dict(size=10, color='green', symbol='star'),
    name='Floor Points',
    yaxis='y1'  # Associate with the first y-axis
)

# Create a scatter plot for obstacle cells
obstacle_trace = go.Scatter(
    x=obstacle_df['x'],
    y=obstacle_df['y'],
    mode='markers',
    marker=dict(size=10, color='red', symbol='square'),
    name='Wall',
    yaxis='y1'  # Associate with the first y-axis
)

# Create layout
layout = go.Layout(
    title='Grid of Points with Floor Points and Obstacles',
    xaxis=dict(title='X'),
    yaxis=dict(title='Y', side='left', showgrid=True),
    showlegend=True
)

# Create a figure and add traces
fig = go.Figure(data=[grid_trace, sample_trace, obstacle_trace], layout=layout)

# Define the start and end points (find nearest nodes among floor points)
start_point = (1, 0.9998)
end_point = (0.999927, 0.999844)

def find_nearest_floor_node(graph, point, floor_points):
    nearest_node = None
    min_distance = float('inf')
    for node in floor_points:
        distance = np.linalg.norm(np.array(node) - np.array(point))
        if distance < min_distance:
            nearest_node = node
            min_distance = distance
    return nearest_node

# Find nearest nodes among floor points
floor_points = [(x, y) for x, y in zip(sample_df['x'], sample_df['y'])]
start_node = find_nearest_floor_node(G, start_point, floor_points)
end_node = find_nearest_floor_node(G, end_point, floor_points)

# Add start and end points to the figure
fig.add_trace(go.Scatter(x=[start_node[0]], y=[start_node[1]], mode='markers', marker=dict(size=10, color='green'), name='Start'))
fig.add_trace(go.Scatter(x=[end_node[0]], y=[end_node[1]], mode='markers', marker=dict(size=10, color='purple'), name='End'))

# Create a graph representation of the grid considering only floor points
G = nx.Graph()

# Add nodes (floor points) to the graph
for node in floor_points:
    G.add_node(node)

# Add edges to connect neighboring floor points (assuming 8-directional movement)
for i in range(len(floor_points)):
    x1, y1 = floor_points[i]
    for j in range(i + 1, len(floor_points)):
        x2, y2 = floor_points[j]
        if abs(x2 - x1) <= resolution and abs(y2 - y1) <= resolution:
            G.add_edge((x1, y1), (x2, y2))

# Use NetworkX's A* algorithm to find the path
path = nx.astar_path(G, start_node, end_node)

# Create a trace for the path
path_trace = go.Scatter(
    x=[point[0] for point in path],
    y=[point[1] for point in path],
    mode='lines',
    line=dict(color='cyan', width=2),
    name='Path'
)

# Add the path trace to the figure
fig.add_trace(path_trace)

# Show the plot
fig.show()


In [92]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import networkx as nx

# Define grid boundaries and resolution
x_min, x_max = data_floor['x'].min(), data_floor['x'].max()
y_min, y_max = data_floor['y'].min(), data_floor['y'].max()
resolution = data_floor['x'][1] - data_floor['x'][2]

# Calculate the number of grid points
num_x_points = int(np.ceil((x_max - x_min) / resolution))
num_y_points = int(np.ceil((y_max - y_min) / resolution))

# Generate the grid
grid_x = []
grid_y = []
for i in range(num_x_points):
    for j in range(num_y_points):
        x_value = x_min + i * resolution
        y_value = y_min + j * resolution
        grid_x.append(x_value)
        grid_y.append(y_value)

# Create a DataFrame for grid points
grid_df = pd.DataFrame({'X': grid_x, 'Y': grid_y})

# Define some sample 'x' and 'y' points
sample_df = data_floor[['x', 'y']]  # Example 'x' and 'y' points

# Create a DataFrame for obstacle positions
obstacle_df = data_wall[['x', 'y']]

# Create a scatter plot for the grid
grid_trace = go.Scatter(
    x=grid_df['X'],
    y=grid_df['Y'],
    mode='markers',
    marker=dict(size=2, color='lightgray'),
    name='Grid Points',
    yaxis='y1'  # Associate with the first y-axis
)

# Create a scatter plot for the sample 'x' and 'y' points
sample_trace = go.Scatter(
    x=sample_df['x'],
    y=sample_df['y'],
    mode='markers',
    marker=dict(size=10, color='green', symbol='star'),
    name='Floor Points',
    yaxis='y1'  # Associate with the first y-axis
)

# Create a scatter plot for obstacle cells
obstacle_trace = go.Scatter(
    x=obstacle_df['x'],
    y=obstacle_df['y'],
    mode='markers',
    marker=dict(size=4, color='red', symbol='star'),
    name='Wall',
    yaxis='y1'  # Associate with the first y-axis
)

# Create layout with a black background
layout = go.Layout(
    title='Grid of Points with Floor Points and Obstacles for Floor 0',
    xaxis=dict(title='X'),
    yaxis=dict(title='Y', side='left', showgrid=True),
    showlegend=True,
    plot_bgcolor='black',  # Set the background color to black
    paper_bgcolor='white'  # Set the paper color to black
)

# Create a figure and add traces
fig = go.Figure(data=[grid_trace, sample_trace, obstacle_trace], layout=layout)

# Define the start and end points (find nearest nodes among floor points)
start_point = (1.000001, 0.99998)
end_point = (0.999922, 0.999829)

def find_nearest_floor_node(graph, point, floor_points):
    nearest_node = None
    min_distance = float('inf')
    for node in floor_points:
        distance = np.linalg.norm(np.array(node) - np.array(point))
        if distance < min_distance:
            nearest_node = node
            min_distance = distance
    return nearest_node

# Find nearest nodes among floor points
floor_points = [(x, y) for x, y in zip(sample_df['x'], sample_df['y'])]
start_node = find_nearest_floor_node(G, start_point, floor_points)
end_node = find_nearest_floor_node(G, end_point, floor_points)

# Add start and end points to the figure
fig.add_trace(go.Scatter(x=[start_node[0]], y=[start_node[1]], mode='markers', marker=dict(size=10, color='yellow'), name='Start'))
fig.add_trace(go.Scatter(x=[end_node[0]], y=[end_node[1]], mode='markers', marker=dict(size=10, color='purple'), name='End'))

# Create a graph representation of the grid considering only floor points
G = nx.Graph()

# Add nodes (floor points) to the graph
for node in floor_points:
    G.add_node(node)

# Add edges to connect neighboring floor points (assuming 8-directional movement)
for i in range(len(floor_points)):
    x1, y1 = floor_points[i]
    for j in range(i + 1, len(floor_points)):
        x2, y2 = floor_points[j]
        if abs(x2 - x1) <= resolution and abs(y2 - y1) <= resolution:
            G.add_edge((x1, y1), (x2, y2))

# Use NetworkX's A* algorithm to find the path
path = nx.astar_path(G, start_node, end_node)

# Create a trace for the path
path_trace = go.Scatter(
    x=[point[0] for point in path],
    y=[point[1] for point in path],
    mode='lines',
    line=dict(color='cyan', width=2),
    name='Path'
)

# Add the path trace to the figure
fig.add_trace(path_trace)

# Show the plot
fig.show()
